# Investigating the Gapminder Systema Globalis Dataset

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

The name "Systema Globalis" is inspired by [Systema Naturae](https://en.wikipedia.org/wiki/Systema_Naturae); the first systematic compilation of all living things from year 1735, by the Swedish Scientist Carl von Linné. The goal of Systema Globalis is to compile all public statistics; Social, Economic and Environmental; into a comparable total dataset.

### Data
This is the main dataset used in tools on the official Gapminder website. It contains local &amp; global statistics combined from hundreds of sources.

For the purposes of this demonstration, we will be using the glob module for file manipulation, pandas and numpy for working with the data, and matplotlib's basic pyplot suite for visualization.

_This exercise is intended to illustrate the choices an analyst must make at each point of the data analysis process, from the initial data-wrangling to the final presentation of findings. The process shown is not being presented as an optimal route. In fact, there will be points along the way where less than optimal choices are made. The intent is to illustrate the potential pitfalls an analyst will encounter when working with an unknown dataset, as well as to demonstrate the shortcomings of certain decisions the novice analyst is likely to make. At these points, suggestions as to how the process could have been improved will be provided and/or the process will be revised and re-performed._ 

In [71]:
import glob
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling

The Systema Globalis data consists of longitudinal country-level data on over 500 indicators of social, economic, health, environmental, and other comparative measures. The data on each indicator is contained in its own csv file, organized by country (geo) and year of measurement (time). 

### General Properties

Due to the multi-file structure of this dataset, analysis involving more than one indicator requires file operations to parse, read, and merge the data from each target indicator. The glob module simplifies this process, allowing for filename 'globbing' based on wildcard searches of filenames. This provides one method of subsetting the files and extracting only the data we are interested in analyzing. 

For example: 

In [59]:
# Returns on the filenames of 'datapoint' csv files containing literacy data:
datafiles_literacy = glob.glob('*datapoints*literacy*.csv')
print(np.array(datafiles_literacy)[:5], '\n\n')

# Returns only the filenames of 'datapoint' csv files containing energy data:
datafiles_energy = glob.glob('*datapoints*energy*.csv')
print(np.array(datafiles_energy)[:5])

['ddf--datapoints--literacy_rate_adult_female_percent_of_females_ages_15_above--by--geo--time.csv'
 'ddf--datapoints--literacy_rate_adult_male_percent_of_males_ages_15_and_above--by--geo--time.csv'
 'ddf--datapoints--literacy_rate_adult_total_percent_of_people_ages_15_and_above--by--geo--time.csv'
 'ddf--datapoints--literacy_rate_youth_female_percent_of_females_ages_15_24--by--geo--time.csv'
 'ddf--datapoints--literacy_rate_youth_male_percent_of_males_ages_15_24--by--geo--time.csv'] 


['ddf--datapoints--energy_from_solid_biofuels_percent--by--geo--time.csv'
 'ddf--datapoints--energy_production_per_person--by--geo--time.csv'
 'ddf--datapoints--energy_production_total--by--geo--time.csv'
 'ddf--datapoints--energy_supply_per_person_toe--by--geo--time.csv'
 'ddf--datapoints--energy_use_per_person--by--geo--time.csv']


In the alternative, we could try reading all of the data into Python, and holding off on trimming it down or subsetting it until after its already been processed.  Due to the size and complexity of the dataset, this will be more computationally intensive than importing only a target subset, but there is a certain appeal to compiling all of the relevant data in a single place, so lets give that a try.  

In [5]:
datafiles_all = glob.glob('*datapoints*.csv')

The gapminder data is particularly well suited to exploratory data analysis because it is largely 'pre-cleaned'. Each table shares a uniform format, making it very easy to read in and merge the data from multiple indicators. 

While data 'wrangling' can be a complex, time-consuming process, especially when dealing with messy data or data from multiple sources, it can be accomplished here with relative ease.

In [60]:
# create empty dictionary into which we will read all of our data files
ref = {}

# loop through filenames, extracting indicator name from filename, and values from each file itself
for filename in datafiles_all:
    key = filename.partition('datapoints--')[2].partition('--')[0]
    data = pd.read_csv(filename)
    # index each table by country and cast longitudinal progression to columns 
    data = data.pivot(index='geo', columns='time')
    # incorporate second-level index (indicator name), necessary when merging multiple indicators 
    data = data.stack(level=0)
    data.index.set_names('indicator', level=1, inplace=True)
    # dictionary entry: key = indicator name, value = reformatted dataframe
    ref[key] = data

We now have a reference dictionary containing the entire gapminder dataset. As mentioned eariler, this "everything in one place" approach is sub-optimal for a number of reasons, one of which will become apparent soon. 

We will finish the process by concatenating all of these individual tables into a single master dataframe. Again, we ensured that this would go smoothly by reindexing and reshaping in the previous step.

In [64]:
master = pd.concat(ref.values())
master = master.sort_index()
# let's take a look at some attributes of our master dataframe
print(master.columns)
print(master.index.names)
print(master.isna().sum().sum())
print(master.count().sum())

Int64Index([1086, 1100, 1279, 1290, 1300, 1309, 1348, 1349, 1351, 1352,
            ...
            2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100],
           dtype='int64', name='time', length=409)
['geo', 'indicator']
33046225
2899967


__Whoa__, this is a bit of a mess. There are datapoints going back all the way to the 11th century, and there are even points out into the future! How is that possible? 

Furthermore, our table contains over 33 million missing values, and fewer than 3 million actual values. In other words, the majority of our dataframe is pretty much useless.

A closer look at our indicators shows that while most of the indicators contain observed values, some are projections. This explains the values for future years. We could fix this by not loading in those particular indicators, or by filtering our data to a specific timeframe. 

How about those missing values though?  Let's take a closer look:

In [69]:
print(master.count())

time
1086       1
1100       1
1279       1
1290       1
1300       1
1309       2
1348       1
1349       1
1351       1
1352       2
1365       2
1374       2
1377       1
1409       2
1413       1
1428       2
1450       1
1459       2
1489       1
1500       7
1545       1
1555       1
1570      11
1575       1
1586       1
1590       1
1600      85
1609      21
1610       1
1614       1
        ... 
2071    1830
2072    1830
2073    1830
2074    1830
2075    8630
2076    1830
2077    1830
2078    1830
2079    1830
2080    8830
2081    1830
2082    1830
2083    1830
2084    1830
2085    8630
2086    1830
2087    1830
2088    1830
2089    1830
2090    8630
2091    1830
2092    1830
2093    1830
2094    1830
2095    8630
2096    1830
2097    1830
2098    1830
2099    2416
2100    8060
Length: 409, dtype: int64


Ok, that explains a lot. If there were no values missing in our table, we would expect to see almost 88,000 observations for each year. Here we're seeing a bunch of years with only 2000 or even just one single value that's a LOT of missing values. 

In [70]:
print(master.count().max())

59140


Even our most populated column contains almost 30,000 missing values. 

### Data Cleaning
In our attempt to put everything in one place, we created a HUGE but mostly empty dataset. The most obvious culprits based on our cursory look are the very early years that contain very few observations, and the future (projection) years which also contain relatively few observations. 

There are some less obvious complications though:
1) Countries whose names change
2) Countries for which very little data has been collected
3) Indicators which only contain data for a limited timeframe

These can each be addressed, whether by 
1) limiting the scope of analysis to only certain countries, 
2) certain indicators, or 
3) certain time-periods 

Each of these solutions moves us away from the 'everything in one place' approach we were initially going for, but it seems like with this dataset it just isn't practical to mash everything together like that. We'll start over and create two new datasets, each applying a hybrid of the filtering methods above.  

In [114]:
# Filtering by time and indicator 
d1 = {}

for filename in datafiles_all:
    if re.search('income|cancer.deaths.*100000', filename):
        key = filename.partition('datapoints--')[2].partition('--')[0]
        data = pd.read_csv(filename)
        # 'pre-cleaning' step to exclude data from earlier than 1950, and exclude projections
        data = data[data.time >= 1950]
        data = data[data.time <= 2018] 
        data = data.pivot(index='geo', columns='time')
        data = data.stack(level=0)
        data.index.set_names('indicator', level=1, inplace=True)
        d1[key] = data

In [ ]:
# Filtering by time and country
d2 = {}

for filename in datafiles_all:
    key = filename.partition('datapoints--')[2].partition('--')[0]
    data = pd.read_csv(filename)
    data = data[data.time >= 1950]
    data = data[data.time <= 2018] 
    data = data.loc[(data.geo == 'usa') | (data.geo == 'ger') | (data.geo == 'chi') |
                    (data.geo == 'jap') | (data.geo == 'aus')]
    data = data.pivot(index='geo', columns='time')
    data = data.stack(level=0)
    data.index.set_names('indicator', level=1, inplace=True)
    d2[key] = data

In [113]:
t1 = pd.concat(d1.values())
t1 = t1.sort_index()
print(t1.shape)
print(t1.isna().sum().sum())
print(t1.count().sum())

t2 = pd.concat(d2.values())
t2 = t2.sort_index()
print(t2.shape)
print(t2.isna().sum().sum())
print(t2.count().sum())

ValueError: No objects to concatenate

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.

## Submitting your Project 

> Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])